# Members
* 107062208邱靖豪 
* 107062132鍾皓崴 
* 107061202蔣承軒 
* 110062577景璞

# Import necessary package

In [1]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import re
from bs4 import BeautifulSoup
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# This function is used to generate the dataset from csv file 

In [2]:
def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

# HTML Preprocessor
We preprocessed some features from original HTML file. Include
* Time
* Category
* Data Channel

# Conver time from string to integer, make it possible to be useful feature

In [3]:
def preprocessor_time(text):
    ts = BeautifulSoup(text, 'html.parser').find_all('time')
    text = ""
    for t in ts:
        text += t.get_text()+ " "
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def generate_time_num_feature(raw_text):
    feature = []
    for r_t in raw_text:
        time_text = preprocessor_time(r_t).split(' ')
        f = []
        for i in range(4):
            try:
                time = int(time_text[i])
            except:
                time = 0
            f.append(time)
        try:
            temp = pd.Timestamp(time_text[0]+'-'+time_text[1]+'-'+time_text[2])
            f.append(temp.dayofweek)
        except:
            f.append(-1)
        f = np.array(f)
        feature.append(f)
    feature = np.array(feature)
    return feature

# This part is used to generate the frequency of specific category
* Find out 5 mostly important features and encode them.

In [4]:
def get_frequency(text_, y):
    cnt = 1
    for label, i in tqdm(zip(y, text_)):
        soup = BeautifulSoup(i)
        for link in soup.find_all('a'):
            t = link.get('href')
            if t != None:
                if t.find('category') != -1 and t.find('http') == -1:
                    for idx, i in enumerate(t.split('/')):
                        if i == 'category': 
                            tag = t.split('/')[idx+1]
                            if tag not in frequency:
                                dict_[tag] = cnt
                                frequency[tag] = 1
                                average_label[tag] = label
                                cnt += 1
                            else:
                                frequency[tag] += 1
                                average_label[tag] += label 


def preprocessor_category_frequency(text_):
    soup = BeautifulSoup(text_)
    text = []
    for link in soup.find_all('a'):
        t = link.get('href')
        if t != None:
            if t.find('category') != -1 and t.find('http') == -1:
                for idx, i in enumerate(t.split('/')):
                    if i == 'category':
                        tag = t.split('/')[idx+1]
                        if tag in frequency and abs(average_label[tag]) > 50:
                            text.append(average_label[tag])
    return text, len(text)


def generate_cate_num_feature(raw_text):
    feature = []
    max_len = -1
    for r_t in tqdm(raw_text):
        href_num, len_ = preprocessor_category_frequency(r_t)
        if max_len < len_:
            max_len = len_
        feature.append(href_num[0:5])
    for idx, r_t in enumerate(feature):
        for i in range(0, 5 - len(r_t)):
            feature[idx].append(0)
    feature = np.array(feature)
    return feature

# Extract the data channel
* Since there are only one data channel tag in each news, so used target embedding to encode it.

In [5]:
def preprocess_datachannel(text_):
    soup = BeautifulSoup(text_)
    pa = soup.find('article')
    text = pa.get('data-channel')
    if text == None:
        text = 'None'
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text 

def get_datachannel_frequency(raw_text, y):
    feature = []
    for r_t in tqdm(raw_text):
        img_num = preprocess_datachannel(r_t)
        feature.append(img_num)
    for tag, label in tqdm(zip(feature, y)):
        if tag not in frequency_data_channel:
            frequency_data_channel[tag] = 1
            data_channel_average[tag] = label
        else:
            frequency_data_channel[tag] += 1
            data_channel_average[tag] += label


def generate_datachannel(raw_text):
    out = []
    for r_t in raw_text:
        text = preprocess_datachannel(r_t)
        if text in frequency_data_channel:
            img_num = data_channel_average[text]
            out.append(img_num)   
        else:
            out.append(-1)
    feature = np.array(out)
    feature = feature.reshape(feature.shape[0], 1)
    return feature

# Model
* Using XGBClassifier as classifier
* Take time, category, channel as feature
* Concate all features to train a model

In [6]:
class numerical_estimator():
    def __init__(self):
        self.pipes = []
        self.pipes.append(generate_time_num_feature)
        self.pipes.append(generate_cate_num_feature)
        self.pipes.append(generate_datachannel)
        self.estimator = XGBClassifier(n_estimators=5000, max_depth=5, gamma=6, objective='binary:logistic', eval_metric='auc')#RandomForestClassifier(max_depth=8, random_state=0, min_samples_leaf=10,min_samples_split=20, n_estimators=1000)
        self.scaler = StandardScaler()
    def fit(self, X, Y):
        
        self.features = []
        for p in self.pipes:
            self.features.append(p(X))
        self.all_feature = np.concatenate(self.features, axis=1)
        print(self.all_feature.shape)
        self.scaler.fit(self.all_feature)
        self.all_feature=self.scaler.transform(self.all_feature)
        self.estimator.fit(self.all_feature, Y)
    def predict(self, X):
        self.features = []
        for p in self.pipes:
            self.features.append(p(X))
        self.all_feature = np.concatenate(self.features, axis=1)
        self.all_feature=self.scaler.transform(self.all_feature)
        print(self.all_feature.shape)
        return self.estimator.predict(self.all_feature)
    def predict_proba(self, X):
        self.features = []
        for p in self.pipes:
            self.features.append(p(X))
        self.all_feature = np.concatenate(self.features, axis=1)
        self.all_feature=self.scaler.transform(self.all_feature)
        print(self.all_feature.shape)
        return self.estimator.predict_proba(self.all_feature)

# Training Process
* Use 22000 data as training data, other are validation data

In [ ]:
stream = get_stream(path='./train.csv', size=22000)
classes = np.array([-1, 1])
batch = next(stream)
X_train, y_train = batch['Page content'], batch['Popularity']

average_label = {}
data_channel_average = {}
frequency = {}
frequency_data_channel = {}
dict_ = {}
get_frequency(X_train, y_train)
get_datachannel_frequency(X_train, y_train)

model = numerical_estimator()
model.fit(X_train, y_train)
score = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])

print('train: {}'.format(score))

def get_stream_from_x(path, size, specific_x):
    count = 0
    chunk_list = []
    for chunk in pd.read_csv(path, chunksize=size, iterator=True):
        if(count < specific_x):
            print(count)
            count+=1
        else:
            chunk_list.append(chunk)
    df = pd.concat(chunk_list)
    return df

batch = get_stream_from_x(path='./train.csv', size=22000, specific_x=1)
X_val, y = batch['Page content'], batch['Popularity']

score = roc_auc_score(y, model.predict_proba(X_val)[:,1])

print('validation: {}'.format(score))

# Write output

In [ ]:
df_test = pd.read_csv('./test.csv')
df_test_pred = pd.DataFrame(columns=['Id', 'Popularity'])
X_test= df_test['Page content']

y_pred = model.predict_proba(X_test)
i = 0
for y in y_pred:
    print(df_test['Id'][i], y[1])
    df_test_pred = df_test_pred.append({'Id': int(df_test['Id'][i]), 'Popularity':y[1]}, ignore_index=True)
    i+=1
df_test_pred['Id'] = df_test_pred['Id'].astype('int')
df_test_pred.to_csv("./num_superXGB_ALL3.csv", index=False)